### GET TWEETS DATA WITH HASGTAG "Macron"

In [ ]:
!pip install twython==3.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from twython import Twython
import json

In [ ]:
# Load credentials from the json file
with open('twitter_credentials.json','r') as file:
    creds = json.load(file)

# instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# creating out query
query = {
    'q': 'Macron',
    'result_type':'popular',
    'count':3000,
    'lang':'en'
}

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# creating a dictionary to put the desired fields
dict_ = {'user':[], 'date':[], 'text':[],'user_location':[], 'favorite_count':[]
         , 'number_followers': [], 'is_retweet':[], 'hashtags':[], 'source':[]}

# set the number of tweets to retrieve
max_tweets = 2000

num_tweets = 0
# loop until retrieve the number of tweets (2000)
while num_tweets <= max_tweets:
    # retrieve the next set of tweets
    search_results = python_tweets.search(**query)['statuses']

    # break out of the loop if there are no more tweets to retrieve
    if len(search_results) == 0:
        break

    # process each tweet in the search results
    for status in search_results:
        # add the tweet data to the dictionary
        dict_['user'].append(status['user']['screen_name'])
        dict_['date'].append(status['created_at'])
        dict_['text'].append(status['text'])
        dict_['user_location'].append(status['user']['location'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['number_followers'].append(status['user']['followers_count'])
        dict_['is_retweet'].append(hasattr(status, 'retweeted_status'))
        
        # extract the text of each hashtag
        hashtags = []
        for hashtag in status['entities']['hashtags']:
            hashtags.append(hashtag['text'])
        dict_['hashtags'].append(hashtags)

        # extract just the name of the application 
        source_html = status['source']
        soup = BeautifulSoup(source_html, 'html.parser')
        source_name = soup.text
        dict_['source'].append(source_name)

        # update the number of tweets retrieved
        num_tweets += 1

        # break out of the loop if the minimum number of tweets retrieved
        if num_tweets >= max_tweets:
            break

# structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)

# df.sort_values(by='favorite_count', inplace=True, ascending=False)

# showing the first 5 rows
df.head(5)

,user,date,text,user_location,favorite_count,number_followers,is_retweet,hashtags,source
0,DrLoupis,Thu Mar 23 21:36:05 +0000 2023,Over a million French protesters gather in Par...,"København, Danmark",50407,285662,False,[],Twitter for iPhone
1,bennyjohnson,Fri Mar 24 01:13:52 +0000 2023,The sum of all fears for globalist technocrats...,Tampa Florida,27384,1391419,False,[],Twitter for iPhone
2,BernieSpofforth,Thu Mar 23 23:01:30 +0000 2023,FRANCE - The firefighters follow the police an...,"North West, England",24930,196109,False,[],Twitter for iPhone
3,BernieSpofforth,Fri Mar 24 12:19:29 +0000 2023,MACRON - Oops the moment Macron realises he’s ...,"North West, England",12298,196109,False,[],Twitter for iPhone
4,DrLoupis,Fri Mar 24 13:28:18 +0000 2023,Millions of people on the streets in Paris. Th...,"København, Danmark",13340,285662,False,[],Twitter for iPhone


In [ ]:
# Exporting data to Macron.csv file
df.to_csv("Macron.csv", index = False)